In [ ]:
from stix2 import MemoryStore, Filter

store = MemoryStore()
store.load_from_file("data/enterprise-attack.json")

In [ ]:
def remove_revoked_deprecated(stix_objects: list) -> list:
    # Note we use .get() because the property may not be present in the JSON data. The default is ""
    # if the property is not set.
    return list(
            filter(
                lambda x: x.get("x_mitre_deprecated", False) is False and x.get("revoked", False) is False, stix_objects
            )
        )

In [ ]:
def get_capec_related(store: MemoryStore, remove_deprecated=False):
    objects = store.query([Filter("type", "=", "attack-pattern"), Filter("external_references.source_name", "=", "capec")])
    if remove_deprecated:
        objects = remove_revoked_deprecated(objects)
    if not objects:
        return []
    return objects


In [ ]:
attack_patterns = get_capec_related(store)
print(len(attack_patterns))

In [ ]:
dump = {}
for pattern in attack_patterns:
    attack_ids = [p.external_id for p in pattern.external_references if p.source_name == "mitre-attack"]
    capec_id = [p.external_id for p in pattern.external_references if p.source_name == "capec"][0]
    dump[capec_id] = attack_ids
    print(f"{pattern.name} -> {capec_id}")


In [ ]:
import json

with open("build/attack-to-capec.json", "w") as f:
    json.dump(dump, f)
